In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_accounts.csv
/kaggle/input/saas-subscription-and-churn-analytics-dataset/README.md
/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_feature_usage.csv
/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_support_tickets.csv
/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_subscriptions.csv
/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_churn_events.csv


In [6]:
# Install DuckDB (SQL engine)
!pip install duckdb --quiet

# Import DuckDB
import duckdb


In [13]:
import duckdb

# Connect to in-memory DuckDB
con = duckdb.connect()

# Load CSV files as SQL tables
con.execute("CREATE TABLE accounts AS SELECT * FROM '/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_accounts.csv'")
con.execute("CREATE TABLE subscriptions AS SELECT * FROM '/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_subscriptions.csv'")
con.execute("CREATE TABLE feature_usage AS SELECT * FROM '/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_feature_usage.csv'")
con.execute("CREATE TABLE support_tickets AS SELECT * FROM '/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_support_tickets.csv'")
con.execute("CREATE TABLE churn_events AS SELECT * FROM '/kaggle/input/saas-subscription-and-churn-analytics-dataset/ravenstack_churn_events.csv'")


In [14]:
# Preview the first 10 rows of the accounts table
con.execute("""
SELECT *
FROM accounts
LIMIT 10
""").fetchdf()


,account_id,account_name,industry,country,signup_date,referral_source,plan_tier,seats,is_trial,churn_flag
0,A-2e4581,Company_0,EdTech,US,2024-10-16,partner,Basic,9,False,False
1,A-43a9e3,Company_1,FinTech,IN,2023-08-17,other,Basic,18,False,True
2,A-0a282f,Company_2,DevTools,US,2024-08-27,organic,Basic,1,False,False
3,A-1f0ac7,Company_3,HealthTech,UK,2023-08-27,other,Basic,24,True,False
4,A-ce550d,Company_4,HealthTech,US,2024-10-27,event,Enterprise,35,False,True
5,A-1b9609,Company_5,EdTech,IN,2023-10-12,ads,Enterprise,4,False,False
6,A-a0ca4e,Company_6,Cybersecurity,US,2024-03-08,ads,Pro,11,False,False
7,A-e5d6ab,Company_7,EdTech,US,2023-04-15,partner,Pro,3,False,False
8,A-7dacce,Company_8,Cybersecurity,CA,2024-09-10,event,Enterprise,12,False,True
9,A-10b8da,Company_9,DevTools,US,2023-05-08,partner,Enterprise,14,False,True


In [16]:
# Active subscribers per month
con.execute("""
WITH months AS (
    -- Generate a list of months based on subscription start dates
    SELECT DISTINCT DATE_TRUNC('month', start_date) AS month
    FROM subscriptions
),
active_subs AS (
    SELECT
        DATE_TRUNC('month', start_date) AS month,
        customer_id
    FROM subscriptions
    WHERE end_date IS NULL OR end_date >= start_date
)
SELECT
    m.month,
    COUNT(DISTINCT a.customer_id) AS active_subscribers
FROM months m
LEFT JOIN active_subs a
    ON DATE_TRUNC('month', a.start_date) <= m.month
GROUP BY m.month
ORDER BY m.month
""").fetchdf()


BinderException: Binder Error: Referenced column "customer_id" not found in FROM clause!
Candidate bindings: "account_id", "subscription_id", "is_trial", "start_date", "auto_renew_flag"

LINE 10:         customer_id
                 ^

In [17]:
con.execute("SELECT * FROM subscriptions LIMIT 5").fetchdf()


,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12,Enterprise,14,2786,33432,False,False,False,True,monthly,True
1,S-0f6f44,A-9b9fe9,2024-06-11,NaT,Pro,17,833,9996,False,False,False,False,monthly,True
2,S-51c0d1,A-659280,2024-11-25,NaT,Enterprise,62,0,0,True,True,False,False,annual,False
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13,Enterprise,5,995,11940,False,False,False,True,monthly,True
4,S-cff5a2,A-ba6516,2024-01-10,NaT,Enterprise,27,5373,64476,False,False,False,False,monthly,True


In [18]:
# Active subscribers per month
con.execute("""
WITH months AS (
    -- Generate all months based on subscription start dates
    SELECT DISTINCT DATE_TRUNC('month', start_date) AS month
    FROM subscriptions
),
active_subs AS (
    SELECT
        account_id,
        start_date,
        end_date
    FROM subscriptions
)
SELECT
    m.month,
    COUNT(DISTINCT a.account_id) AS active_subscribers
FROM months m
LEFT JOIN active_subs a
    ON a.start_date <= m.month
    AND (a.end_date IS NULL OR a.end_date >= m.month)
GROUP BY m.month
ORDER BY m.month
""").fetchdf()


,month,active_subscribers
0,2023-01-01,0
1,2023-02-01,2
2,2023-03-01,10
3,2023-04-01,21
4,2023-05-01,37
5,2023-06-01,50
6,2023-07-01,71
7,2023-08-01,90
8,2023-09-01,117
9,2023-10-01,131


In [19]:
# Monthly churn rate
con.execute("""
WITH months AS (
    SELECT DISTINCT DATE_TRUNC('month', start_date) AS month
    FROM subscriptions
),
active_accounts AS (
    SELECT
        account_id,
        start_date,
        end_date
    FROM subscriptions
),
churned_accounts AS (
    SELECT
        account_id,
        DATE_TRUNC('month', churn_date) AS churn_month
    FROM churn_events
)
SELECT
    m.month,
    COUNT(DISTINCT c.account_id) AS churned_accounts,
    COUNT(DISTINCT a.account_id) AS active_accounts_start,
    ROUND(
        100.0 * COUNT(DISTINCT c.account_id) / NULLIF(COUNT(DISTINCT a.account_id),0),
        2
    ) AS churn_rate_percent
FROM months m
LEFT JOIN active_accounts a
    ON a.start_date <= m.month
    AND (a.end_date IS NULL OR a.end_date >= m.month)
LEFT JOIN churned_accounts c
    ON c.churn_month = m.month
GROUP BY m.month
ORDER BY m.month
""").fetchdf()


,month,churned_accounts,active_accounts_start,churn_rate_percent
0,2023-01-01,1,0,NaN
1,2023-02-01,0,2,0.00
2,2023-03-01,5,10,50.00
3,2023-04-01,3,21,14.29
4,2023-05-01,3,37,8.11
5,2023-06-01,5,50,10.00
6,2023-07-01,6,71,8.45
7,2023-08-01,6,90,6.67
8,2023-09-01,6,117,5.13
9,2023-10-01,10,131,7.63


In [20]:
# Plan-level churn rate (overall)
con.execute("""
WITH active_accounts AS (
    SELECT
        account_id,
        plan_tier,
        start_date,
        end_date
    FROM subscriptions
),
churned_accounts AS (
    SELECT
        ce.account_id,
        s.plan_tier
    FROM churn_events ce
    LEFT JOIN subscriptions s
        ON ce.account_id = s.account_id
)
SELECT
    a.plan_tier,
    COUNT(DISTINCT c.account_id) AS churned_accounts,
    COUNT(DISTINCT a.account_id) AS active_accounts,
    ROUND(
        100.0 * COUNT(DISTINCT c.account_id) / NULLIF(COUNT(DISTINCT a.account_id),0),
        2
    ) AS churn_rate_percent
FROM active_accounts a
LEFT JOIN churned_accounts c
    ON a.account_id = c.account_id
GROUP BY a.plan_tier
ORDER BY churn_rate_percent DESC
""").fetchdf()


,plan_tier,churned_accounts,active_accounts,churn_rate_percent
0,Basic,338,480,70.42
1,Enterprise,337,479,70.35
2,Pro,332,474,70.04


In [21]:
# Feature adoption per month
con.execute("""
WITH usage_month AS (
    SELECT
        account_id,
        feature_name,
        DATE_TRUNC('month', usage_date) AS month
    FROM feature_usage
)
SELECT
    month,
    feature_name,
    COUNT(DISTINCT account_id) AS active_users
FROM usage_month
GROUP BY month, feature_name
ORDER BY month, active_users DESC
""").fetchdf()


BinderException: Binder Error: Referenced column "account_id" not found in FROM clause!
Candidate bindings: "usage_id", "usage_count", "subscription_id", "error_count", "usage_duration_secs"

LINE 4:         account_id,
                ^

In [22]:
# Feature adoption per month (using subscription_id -> account_id)
con.execute("""
WITH usage_with_account AS (
    SELECT
        fu.subscription_id,
        s.account_id,
        DATE_TRUNC('month', fu.usage_date) AS month,
        fu.feature_name
    FROM feature_usage fu
    LEFT JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
)
SELECT
    month,
    feature_name,
    COUNT(DISTINCT account_id) AS active_users
FROM usage_with_account
GROUP BY month, feature_name
ORDER BY month, active_users DESC
""").fetchdf()


,month,feature_name,active_users
0,2023-01-01,feature_22,36
1,2023-01-01,feature_26,33
2,2023-01-01,feature_1,33
3,2023-01-01,feature_10,32
4,2023-01-01,feature_40,31
...,...,...,...
955,2024-12-01,feature_18,22
956,2024-12-01,feature_33,22
957,2024-12-01,feature_3,20
958,2024-12-01,feature_24,19


In [23]:
# Support tickets vs churn
con.execute("""
WITH tickets_per_account AS (
    SELECT
        s.account_id,
        COUNT(st.subscription_id) AS tickets_count
    FROM support_tickets st
    LEFT JOIN subscriptions s
        ON st.subscription_id = s.subscription_id
    GROUP BY s.account_id
),
churn_status AS (
    SELECT
        account_id,
        1 AS churned
    FROM churn_events
)
SELECT
    t.tickets_count,
    COUNT(t.account_id) AS total_accounts,
    SUM(COALESCE(c.churned, 0)) AS churned_accounts,
    ROUND(100.0 * SUM(COALESCE(c.churned, 0)) / COUNT(t.account_id), 2) AS churn_rate_percent
FROM tickets_per_account t
LEFT JOIN churn_status c
    ON t.account_id = c.account_id
GROUP BY t.tickets_count
ORDER BY tickets_count
""").fetchdf()


BinderException: Binder Error: Table "st" does not have a column named "subscription_id"

Candidate bindings: : "account_id"

LINE 8:         ON st.subscription_id = s.subscription_id
                   ^

In [24]:
# Support tickets vs churn (corrected)
con.execute("""
WITH tickets_per_account AS (
    SELECT
        account_id,
        COUNT(*) AS tickets_count
    FROM support_tickets
    GROUP BY account_id
),
churn_status AS (
    SELECT
        account_id,
        1 AS churned
    FROM churn_events
)
SELECT
    t.tickets_count,
    COUNT(t.account_id) AS total_accounts,
    SUM(COALESCE(c.churned, 0)) AS churned_accounts,
    ROUND(100.0 * SUM(COALESCE(c.churned, 0)) / COUNT(t.account_id), 2) AS churn_rate_percent
FROM tickets_per_account t
LEFT JOIN churn_status c
    ON t.account_id = c.account_id
GROUP BY t.tickets_count
ORDER BY tickets_count
""").fetchdf()


,tickets_count,total_accounts,churned_accounts,churn_rate_percent
0,1,37,25.0,67.57
1,2,110,91.0,82.73
2,3,158,129.0,81.65
3,4,150,124.0,82.67
4,5,150,126.0,84.00
5,6,65,52.0,80.00
6,7,37,23.0,62.16
7,8,20,16.0,80.00
8,9,8,4.0,50.00
9,10,1,1.0,100.00
